In [1]:
import pickle
from datasets.pipelines import PreNormalize3D
from utils.visualize import Vis3DPose

In [2]:
annotations = './data/nturgbd/ntu60_3danno.pkl'
with open(annotations, "rb") as f:
    data = pickle.load(f)

In [3]:
anno = data['annotations'][0]
anno = PreNormalize3D()(anno)

AttributeError: can't set attribute

Error in callback <function _draw_all_if_interactive at 0x7f242ff613a0> (for post_execute):


AttributeError: can't set attribute

<Figure size 800x800 with 0 Axes>